In [1]:
%matplotlib inline
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, linewidth=75)

In [2]:
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4])
t_un = 0.1 * t_u

In [3]:
def model(t_u, w, b):
    return w * t_u + b

In [4]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c) ** 2
    return squared_diffs.mean()

In [5]:
import torch.optim as optim

dir(optim)

['ASGD',
 'Adadelta',
 'Adafactor',
 'Adagrad',
 'Adam',
 'AdamW',
 'Adamax',
 'LBFGS',
 'NAdam',
 'Optimizer',
 'RAdam',
 'RMSprop',
 'Rprop',
 'SGD',
 'SparseAdam',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_adafactor',
 '_functional',
 'lr_scheduler',
 'swa_utils']

In [6]:
# パラメータ初期化（w=1.0, b=0.0）および勾配追跡の有効化
params = torch.tensor([1.0, 0.0], requires_grad=True)

# 学習率の設定
learning_rate = 1e-5

# PyTorchのSGD（確率的勾配降下法）オプティマイザを初期化
# params をリストとして渡すことで最適化対象を指定
optimizer = optim.SGD([params], lr=learning_rate)

In [7]:
# モデルによる予測値を計算（線形モデル t_p = w * t_u + b）
t_p = model(t_u, *params)

# 予測値と正解データ（摂氏）との誤差を損失関数で計算（平均二乗誤差）
loss = loss_fn(t_p, t_c)

# 損失に基づいてパラメータ（w, b）に対する勾配を自動計算
loss.backward()

# 勾配に基づいて optimizer がパラメータを更新（学習ステップ）
optimizer.step()

# 更新後のパラメータの確認
params

tensor([ 9.5483e-01, -8.2600e-04], requires_grad=True)

In [8]:
# パラメータの初期化（w = 1.0, b = 0.0）と勾配追跡の有効化
params = torch.tensor([1.0, 0.0], requires_grad=True)

# 学習率を設定してSGDオプティマイザを作成（パラメータはリストで渡す）
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

# モデルによる予測値の計算（t_p = w * t_un + b）
t_p = model(t_un, *params)

# 損失関数により予測値と正解との誤差（MSE）を計算
loss = loss_fn(t_p, t_c)

# オプティマイザに保持されている既存の勾配情報を初期化（累積防止）
optimizer.zero_grad()

# 誤差に基づいてパラメータ（params）に対する勾配を自動計算
loss.backward()

# 勾配に基づいてSGDによるパラメータの更新を実行
optimizer.step()

# 更新後のパラメータを確認
params

tensor([1.7761, 0.1064], requires_grad=True)

In [9]:
# モデルの訓練ループ関数（勾配降下法 + オプティマイザ使用）
def training_loop(n_epochs, optimizer, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        # モデルによる予測（forward pass）
        t_p = model(t_u, *params)

        # 予測と正解の誤差（損失）を計算
        loss = loss_fn(t_p, t_c)

        # 勾配の初期化（累積防止）
        optimizer.zero_grad()

        # 損失に基づいてパラメータの勾配を計算（backward pass）
        loss.backward()

        # 勾配に基づいてパラメータを更新（optimizerによるstep）
        optimizer.step()

        # 500エポックごとに現在の損失を出力
        if epoch % 500 == 0:
            print("Epoch %d, Loss %f" % (epoch, float(loss)))

    # 最終的なパラメータを返す
    return params

In [10]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)  # <1>

training_loop(
    n_epochs=5000, optimizer=optimizer, params=params, t_u=t_un, t_c=t_c  # <1>
)

Epoch 500, Loss 7.860119
Epoch 1000, Loss 3.828538
Epoch 1500, Loss 3.092191
Epoch 2000, Loss 2.957697
Epoch 2500, Loss 2.933134
Epoch 3000, Loss 2.928648
Epoch 3500, Loss 2.927831
Epoch 4000, Loss 2.927680
Epoch 4500, Loss 2.927652
Epoch 5000, Loss 2.927647


tensor([  5.3671, -17.3012], requires_grad=True)

In [11]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-1
optimizer = optim.Adam([params], lr=learning_rate)  # <1>

training_loop(
    n_epochs=2000, optimizer=optimizer, params=params, t_u=t_u, t_c=t_c  # <2>
)

Epoch 500, Loss 7.612898
Epoch 1000, Loss 3.086699
Epoch 1500, Loss 2.928580
Epoch 2000, Loss 2.927644


tensor([  0.5367, -17.3021], requires_grad=True)

In [12]:
n_samples = t_u.shape[0]  # サンプル数（全体のデータ数）を取得
n_val = int(0.2 * n_samples)  # 検証用に全体の20%を確保

shuffled_indices = torch.randperm(
    n_samples
)  # データのインデックスをランダムにシャッフル

# トレーニング用：最後の n_val 件を除いたインデックス
train_indices = shuffled_indices[:-n_val]

# 検証用：最後の n_val 件
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices  # 各インデックスを確認

(tensor([10,  1,  4,  5,  2,  6,  3,  8,  0]), tensor([7, 9]))

In [13]:
# トレーニングデータと検証データに温度入力と正解値を分割
train_t_u = t_u[train_indices]  # トレーニング用の温度（華氏）
train_t_c = t_c[train_indices]  # トレーニング用の温度（摂氏）

val_t_u = t_u[val_indices]  # 検証用の温度（華氏）
val_t_c = t_c[val_indices]  # 検証用の温度（摂氏）

# 正規化処理：入力温度を0.1倍してスケーリング
train_t_un = 0.1 * train_t_u
val_t_un = 0.1 * val_t_u

In [14]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u, train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params)  # <1>
        train_loss = loss_fn(train_t_p, train_t_c)

        val_t_p = model(val_t_u, *params)  # <1>
        val_loss = loss_fn(val_t_p, val_t_c)

        optimizer.zero_grad()
        train_loss.backward()  # <2>
        optimizer.step()

        if epoch <= 3 or epoch % 500 == 0:
            print(
                f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                f" Validation loss {val_loss.item():.4f}"
            )

    return params

In [15]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs=3000,
    optimizer=optimizer,
    params=params,
    train_t_u=train_t_un,  # <1>
    val_t_u=val_t_un,  # <1>
    train_t_c=train_t_c,
    val_t_c=val_t_c,
)

Epoch 1, Training loss 88.5971, Validation loss 43.3170
Epoch 2, Training loss 34.4219, Validation loss 35.0349
Epoch 3, Training loss 27.5799, Validation loss 40.2142
Epoch 500, Training loss 9.5169, Validation loss 9.0298
Epoch 1000, Training loss 4.5432, Validation loss 2.5969
Epoch 1500, Training loss 3.1109, Validation loss 2.9066
Epoch 2000, Training loss 2.6984, Validation loss 4.1562
Epoch 2500, Training loss 2.5796, Validation loss 5.1387
Epoch 3000, Training loss 2.5454, Validation loss 5.7558


tensor([  5.6473, -18.7334], requires_grad=True)

In [16]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u, train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p, train_t_c)

        with torch.no_grad():  # <1>
            val_t_p = model(val_t_u, *params)
            val_loss = loss_fn(val_t_p, val_t_c)
            assert val_loss.requires_grad == False  # <2>

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

In [17]:
def calc_forward(t_u, t_c, is_train):
    # 学習時は勾配を有効にし、検証時は無効にする（メモリ節約と高速化）
    with torch.set_grad_enabled(is_train):
        t_p = model(t_u, *params)  # 予測値を計算
        loss = loss_fn(t_p, t_c)  # 損失を計算
    return loss  # 損失を返す